# Prediction
## Part A

# Import packages

In [1]:
%matplotlib agg

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential, layers
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import random
import math
import matplotlib as mpl
from matplotlib import pyplot as plt
from copy import deepcopy
mpl.interactive = 'off'
mpl.style.use('fivethirtyeight')

/Users/sarahcoopmans96/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Import data

In [3]:
operational_settings = ['operational_setting_{}'.format(i + 1) for i in range (3)]
sensor_columns = ['sensor_measurement_{}'.format(i + 1) for i in range(21)]
cols = ['engine_no', 'time_in_cycles'] + operational_settings + sensor_columns + ['empty1', 'empty2']
df_train = pd.read_csv('DataTrain.txt', sep=' ', names=cols)

# Because the .txt files ends every line with two spaces, pandas interprets these as two empty columns, so these can be dropped.
df_train = df_train.drop(columns=['empty1','empty2'])

In [4]:
df_train.tail()

,engine_no,time_in_cycles,operational_setting_1,operational_setting_2,operational_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,...,sensor_measurement_12,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,...,519.49,2388.26,8137.60,8.4956,0.03,397,2388,100.0,38.49,22.9735
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,...,519.68,2388.22,8136.50,8.5139,0.03,395,2388,100.0,38.30,23.1594
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,...,520.01,2388.24,8141.05,8.5646,0.03,398,2388,100.0,38.44,22.9333
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,...,519.67,2388.23,8139.29,8.5389,0.03,395,2388,100.0,38.29,23.0640
20630,100,200,-0.0032,-0.0005,100.0,518.67,643.85,1600.38,1432.14,14.62,...,519.30,2388.26,8137.33,8.5036,0.03,396,2388,100.0,38.37,23.0522


# Add RUL as a column

In [8]:
def find_nr_cycles(engine):
    df_engine_no = df_train['engine_no'] == engine
    return df_train[df_engine_no].shape[0]

In [9]:
result = []
for engine in range(1, 101):
    for cycle in reversed(range(find_nr_cycles(engine))):
        result.append(cycle)
        
df_train['RUL'] = result

In [10]:
df_train.head()

,engine_no,time_in_cycles,operational_setting_1,operational_setting_2,operational_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,...,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


# Feature selection and Data Exploration

### Plotting

In [11]:
def draw_plot():
    fig, axes = plt.subplots(3,8, sharex=True)
    fig.set_figwidth(25)
    fig.set_figheight(10)
    i = 2
    for row in axes:
        for ax in row:
            ax.scatter(x=df_train.iloc[:,26] , y=df_train.iloc[:,i], alpha=0.5)
            ax.set_xlabel("RUL")
            ax.set_ylabel(df_train.columns[i])
            i += 1
    plt.tight_layout()
    fig.savefig('Data_exploration_figure')
    
draw_plot()

Based on the plots, we decided to do a visual feature selection, since there is a very clear distinction between the variables which have a correlation with RUL and those that don't.

#### Variables that will be dropped:
- operational_settings_2
- operational_settings_3
- sensor_measurements_1
- sensor_measurements_5
- sensor_measurements_6
- sensor_measurements_10
- sensor_measurements_16
- sensor_measurements_18
- sensor_measurements_19


In [12]:
dropped_variables = ["operational_setting_2", "operational_setting_3", "sensor_measurement_1", "sensor_measurement_5", "sensor_measurement_6", "sensor_measurement_10", "sensor_measurement_16", "sensor_measurement_18", "sensor_measurement_19"]

df_train = df_train.drop(columns=dropped_variables)

df_train.head()

,engine_no,time_in_cycles,operational_setting_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_7,sensor_measurement_8,sensor_measurement_9,sensor_measurement_11,sensor_measurement_12,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_17,sensor_measurement_20,sensor_measurement_21,RUL
0,1,1,-0.0007,641.82,1589.70,1400.60,554.36,2388.06,9046.19,47.47,521.66,2388.02,8138.62,8.4195,392,39.06,23.4190,191
1,1,2,0.0019,642.15,1591.82,1403.14,553.75,2388.04,9044.07,47.49,522.28,2388.07,8131.49,8.4318,392,39.00,23.4236,190
2,1,3,-0.0043,642.35,1587.99,1404.20,554.26,2388.08,9052.94,47.27,522.42,2388.03,8133.23,8.4178,390,38.95,23.3442,189
3,1,4,0.0007,642.35,1582.79,1401.87,554.45,2388.11,9049.48,47.13,522.86,2388.08,8133.83,8.3682,392,38.88,23.3739,188
4,1,5,-0.0019,642.37,1582.85,1406.22,554.00,2388.06,9055.15,47.28,522.19,2388.04,8133.80,8.4294,393,38.90,23.4044,187


# Create train/test split

In [13]:
def get_dataframe_for(engine):
    engine_no = df_train['engine_no'] == engine
    return df_train[engine_no]

In [14]:
#80 train engines (80%)
train_engines = sorted(random.sample(list(set(df_train['engine_no'])), 80))

#20 test engines (20%)
test_engines = list(set(df_train['engine_no']))
for i in train_engines:
    test_engines.remove(i)

print(train_engines)
print(test_engines)

[1, 2, 3, 4, 6, 9, 10, 11, 13, 15, 16, 17, 19, 20, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 79, 81, 82, 86, 87, 88, 89, 90, 92, 93, 94, 95, 96, 98, 99, 100]
[5, 7, 8, 12, 14, 18, 21, 22, 25, 43, 59, 62, 70, 78, 80, 83, 84, 85, 91, 97]


## Create test and train dataframes

In [15]:
df_training = get_dataframe_for(train_engines[0]).reset_index(drop=True)

for engine in train_engines[1:]:
    df_training = df_training.append(get_dataframe_for(engine).reset_index(drop=True))
    
df_training.shape

(16525, 18)

In [16]:
df_testing = get_dataframe_for(test_engines[0]).reset_index(drop=True)

for engine in test_engines[1:]:
    df_testing = df_testing.append(get_dataframe_for(engine).reset_index(drop=True))

df_testing.shape

(4106, 18)

In [17]:
df_testing.head()

,engine_no,time_in_cycles,operational_setting_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_7,sensor_measurement_8,sensor_measurement_9,sensor_measurement_11,sensor_measurement_12,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_17,sensor_measurement_20,sensor_measurement_21,RUL
0,5,1,0.0031,641.77,1583.59,1395.26,554.39,2387.98,9066.49,47.19,522.34,2388.00,8152.22,8.4102,393,38.98,23.4468,268
1,5,2,0.0002,642.42,1589.69,1394.88,555.57,2388.01,9063.03,47.09,522.60,2387.99,8153.04,8.4053,392,39.12,23.4075,267
2,5,3,0.0002,642.07,1582.45,1396.29,554.49,2388.01,9068.21,47.11,522.85,2388.02,8148.42,8.3990,393,39.18,23.3298,266
3,5,4,0.0017,642.08,1586.84,1397.94,555.16,2387.96,9070.56,47.29,522.11,2387.99,8148.81,8.3876,390,39.22,23.4306,265
4,5,5,-0.0007,642.03,1581.45,1394.28,554.59,2387.98,9069.25,47.09,522.60,2387.95,8147.69,8.4129,390,39.17,23.3331,264


In [18]:
def evaluate_model(x_test, y_test, model):
    y_pred = model.predict(x_test)
    
    errors = abs(y_pred-y_test)
    errors_2 = (y_pred-y_test)**2
    errors_p = abs(y_pred-y_test)/(y_test+0.5)
    y_test = y_test.reshape([-1])
    y_pred = y_pred.reshape([-1])
    errordf = pd.DataFrame(dict(real=y_test, predicted=y_pred))
    print('OVER ALL DATAPOINTS:')
    print('Mean Absolute Error:', round(np.mean(errors), 2))
    print('Mean Squared Error:', round(np.mean(errors_2), 2))
    print('Mean Absolute Percentage Error: %i%%' % round(np.mean(errors_p)*100, 2))
    print()
    
    relevant = errordf[errordf['real'] <= 40]
    
    rel_errors = abs(relevant['predicted']-relevant['real'])
    rel_errors_2 = (relevant['predicted']-relevant['real'])**2
    rel_errors_p = abs(relevant['predicted']-relevant['real'])/(relevant['real']+0.5)
    print("OVER ONLY THE MOST RELEVANT RUL'S:")
    print('Mean Absolute Error:', round(np.mean(rel_errors), 2))
    print('Mean Squared Error:', round(np.mean(rel_errors_2), 2))
    print('Mean Absolute Percentage Error: %i%%' % round(np.mean(rel_errors_p)*100, 2))
    
    return y_pred

# Fit RandomForest using engine-based test/train split

In [19]:
train_features = np.array(df_training.iloc[:,:-1])

train_labels = np.array(df_training['RUL'])

In [20]:
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)

In [21]:
rf.fit(train_features, train_labels)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [22]:
test_features = np.array(df_testing.iloc[:,:-1])

test_labels = np.array(df_testing['RUL'])

In [23]:
rf_predictions = evaluate_model(test_features, test_labels, rf)

OVER ALL DATAPOINTS:
Mean Absolute Error: 34.97
Mean Squared Error: 2817.98
Mean Absolute Percentage Error: 40%

OVER ONLY THE MOST RELEVANT RUL'S:
Mean Absolute Error: 7.22
Mean Squared Error: 130.32
Mean Absolute Percentage Error: 61%


# Linear Regression

In [24]:
lm = LinearRegression()

In [25]:
x_train = np.array(df_training.iloc[:,1:-1])
y_train = np.array(df_training.iloc[:,-1])
x_test = np.array(df_testing.iloc[:,1:-1])
y_test = np.array(df_testing.iloc[:,-1])

In [26]:
lm.fit(X=x_train, y=y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [27]:
lin_y_pred = evaluate_model(x_test, y_test, lm)

OVER ALL DATAPOINTS:
Mean Absolute Error: 25.31
Mean Squared Error: 1021.49
Mean Absolute Percentage Error: 78%

OVER ONLY THE MOST RELEVANT RUL'S:
Mean Absolute Error: 21.11
Mean Squared Error: 617.21
Mean Absolute Percentage Error: 296%


# LSTM

Function below builds the 3D tensors required for an LSTM. First scales the values to the range 0-1, for better performance of NN. Then iterates over each engine, and takes a segment of *x* timesteps as inputs to LSTM. Optionally takes a drop_zeros argument. This drops the timeframes where the label=0. because 0's confuse the MAPE loss function.

In [28]:
#Create tensor and ylabels for training
def build_lstm_data(dataframe, timesteps, skip=1, scaler=None, drop_zeros=False):
    xresult = []
    yresult = []
    
    #Scale data to values between 0 and 1
    df = deepcopy(dataframe)
    if scaler is None:
        scaler = MinMaxScaler()
        df.iloc[:,:-1] = scaler.fit_transform(dataframe.iloc[:,:-1])
    else:
        df.iloc[:,:-1] = scaler.transform(dataframe.iloc[:,:-1])
    
    #Transform DF to 3D-array for LSTM
    for _, engineframe in df.groupby('engine_no'):
        for i in range(engineframe.shape[0]-timesteps):
            if not (drop_zeros and engineframe.iloc[i+timesteps, -1] == 0):
                xresult.append(np.array(engineframe.iloc[i:i+timesteps:skip, 1:-1]))
                yresult.append(engineframe.iloc[i+timesteps, -1])
    
    return np.transpose(np.dstack(xresult), (2,0,1)), np.array(yresult).reshape((-1,1)), scaler

In [29]:
def build_LSTM(loss_function):
    model = Sequential()
    model.add(layers.LSTM(20, activation='relu', input_shape=(20,16), return_sequences=True))
    model.add(layers.LSTM(20, activation='relu'))
    model.add(layers.Dense(20, activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=loss_function)
    print(model.summary())
    return model

In [30]:
#Create training sets for the MAE loss function and MAPE loss function separately.
x_train_mape, y_train_mape, scaler = build_lstm_data(df_training, 20, drop_zeros=True)
x_train_mae, y_train_mae, scaler = build_lstm_data(df_training, 20, drop_zeros=False)

/Users/sarahcoopmans96/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/sarahcoopmans96/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [31]:
#Create test sets
x_test, y_test, scaler = build_lstm_data(df_testing, 20, scaler=scaler)

In [32]:
mae_model = build_LSTM('mae')
mae_model.fit(x=x_train_mae, y=y_train_mae, epochs=16, batch_size=100, validation_split=0.1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 20)            2960      
_________________________________________________________________
lstm_1 (LSTM)                (None, 20)                3280      
_________________________________________________________________
dense (Dense)                (None, 20)                420       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 21        
Total params: 6,681
Trainable params: 6,681
Non-trainable params: 0
_________________________________________________________________
None
Train on 13432 samples, validate on 1493 samples
Epoch 1/16
13432/13432 [==============================] - 10s 737us/step - loss: 49.3996 - val_loss: 40.2233
Epoch 2/16
13432/13432 [==============================] - 9s 674us/step - loss: 27.7388 - val_loss: 42.8630
E

In [33]:
mape_model = build_LSTM('mape')
mape_model.fit(x=x_train_mape, y=y_train_mape, epochs=16, batch_size=100, validation_split=0.1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 20, 20)            2960      
_________________________________________________________________
lstm_3 (LSTM)                (None, 20)                3280      
_________________________________________________________________
dense_2 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21        
Total params: 6,681
Trainable params: 6,681
Non-trainable params: 0
_________________________________________________________________
None
Train on 13360 samples, validate on 1485 samples
Epoch 1/16
13360/13360 [==============================] - 8s 626us/step - loss: 76.5319 - val_loss: 46.5058
Epoch 2/16
13360/13360 [==============================] - 6s 467us/step - loss: 38.3817 - val_loss: 46.8038
Ep

*As expected: the following statistics show a clear preference for MAPE based LSTM in terms of error in the most relevant region of predictions (RUL <40)*

In [34]:
y_pred_mae = evaluate_model(x_test, y_test, mae_model)

OVER ALL DATAPOINTS:
Mean Absolute Error: 20.49
Mean Squared Error: 770.62
Mean Absolute Percentage Error: 24%

OVER ONLY THE MOST RELEVANT RUL'S:
Mean Absolute Error: 5.94
Mean Squared Error: 57.79
Mean Absolute Percentage Error: 35%


In [35]:
y_pred_mape = evaluate_model(x_test, y_test, mape_model)

OVER ALL DATAPOINTS:
Mean Absolute Error: 19.51
Mean Squared Error: 893.94
Mean Absolute Percentage Error: 22%

OVER ONLY THE MOST RELEVANT RUL'S:
Mean Absolute Error: 4.32
Mean Squared Error: 40.68
Mean Absolute Percentage Error: 34%


##### The effect of which can also be inspected in the following plot

In [36]:
fig, ax = plt.subplots()
ax.set_xlim(-1,25.5)
ax.set_ylim(-1,45.5)
fig.set_figheight(7)
fig.set_figwidth(10)
ax.set_title('Comparison of MAE-based and MAPE-based LSTM results.')
ax.set_xlabel('Ground truth RUL values')
ax.set_ylabel('Predicted RUL values')
ax.scatter(x=y_test, y=y_pred_mae, color='red', s=50, alpha=1, label='MAE predictions')
ax.scatter(x=y_test, y=y_pred_mape, s=50, alpha=0.7, label='MAPE predictions')
ax.plot(y_test, y_test, color='black', linewidth=1, alpha=0.8, label='Correct predictions')
ax.legend()
plt.tight_layout()
fig.savefig('mae_mape full comparison')   
plt.close(fig)

# Prediction
## Part B 

In [37]:
operational_settings = ['operational_setting_{}'.format(i + 1) for i in range (3)]
sensor_columns = ['sensor_measurement_{}'.format(i + 1) for i in range(21)]
cols = ['engine_no', 'time_in_cycles'] + operational_settings + sensor_columns + ['empty1', 'empty2']
df_schedule = pd.read_csv('DataSchedule.txt', sep=' ', names=cols)

# Because the .txt files ends every line with two spaces, pandas interprets these as two empty columns, so these can be dropped.
df_schedule = df_schedule.drop(columns=['empty1','empty2'])

In [38]:
df_schedule.head()

,engine_no,time_in_cycles,operational_setting_1,operational_setting_2,operational_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,...,sensor_measurement_12,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130


In [39]:
def build_lstm_prediction_data(dataframe, timesteps, scaler=None, skip=1):
    xresult = []
    yresult = []
    
    dataframe = dataframe.drop(columns=dropped_variables)
    
    #Scale data to values between 0 and 1
    df = pd.DataFrame(scaler.transform(dataframe.iloc[:,:]))
    
    #Transform DF to 3D-array for LSTM
    for _, engineframe in df.groupby(0):
        xresult.append(np.array(engineframe.iloc[-timesteps:, 1:]))
    
    return np.transpose(np.dstack(xresult), (2,0,1))

In [40]:
prediction_data = build_lstm_prediction_data(df_schedule, 20, scaler=scaler)

In [41]:
predictions = mape_model.predict(prediction_data)

In [42]:
predictions = predictions.round()

In [43]:
# Export predictions to Excel
rul_predictions = pd.DataFrame({'id': range(1,101), 'RUL': predictions[:,0]})
rul_predictions.to_excel('RUL_predictions.xlsx')